In [221]:
import pandas as pd
import numpy as np
import psycopg2
from utils.db_connect import create_db_connection
from sqlalchemy import create_engine

In [222]:
conn = create_db_connection(host='localhost')

count sensors

tickets

tickets correct

In [223]:
# count tickets for each trip
conn.rollback()
query = """
SELECT 
    COUNT(fare) as passengers_in,
    trip_id,
    MAX(school) as school,
    MAX(hospital) as hospital,
    MAX(peak_hour) as peak_hour,
    DATE_TRUNC('day', timestamp) as day,
    MAX(timestamp) as timestamp
FROM 
    raw_tickets 
GROUP BY 
    trip_id, 
    DATE_TRUNC('day', timestamp),
    DATE_TRUNC('month', timestamp),
    DATE_TRUNC('year', timestamp)
ORDER BY 
    day
"""

tickets = pd.read_sql(query, conn)
tickets.head()


,passengers_in,trip_id,school,hospital,peak_hour,day,timestamp
0,514,0004188642024090920250612,0,0,1,2025-05-17,2025-05-17 17:33:00
1,870,0004188652024090920250612,0,0,1,2025-05-17,2025-05-17 18:33:00
2,814,0004188662024090920250612,0,0,1,2025-05-17,2025-05-17 19:33:00
3,460,0004188672024090920250612,0,0,1,2025-05-17,2025-05-17 20:33:00
4,354,0004188682024090920250612,0,0,1,2025-05-17,2025-05-17 12:44:00


In [224]:
print(tickets.shape)

(3376, 7)


In [225]:
# count people out for each trip
conn.rollback()
query = """
SELECT 
    COUNT(status) as passengers_out,
    trip_id,
    DATE_TRUNC('day', timestamp) as day,
    MAX(timestamp) as timestamp
FROM 
    raw_sensors
GROUP BY 
    trip_id, 
    DATE_TRUNC('day', timestamp),
    DATE_TRUNC('month', timestamp),
    DATE_TRUNC('year', timestamp)
ORDER BY 
    day
"""

sensors =pd.read_sql(query, conn)
sensors.head()

,passengers_out,trip_id,day,timestamp
0,90,0004211282024090920250612,2025-05-17,2025-05-17 17:56:00
1,112,0004246792025030620250612,2025-05-17,2025-05-17 21:18:00
2,148,0004245642025030620250612,2025-05-17,2025-05-17 11:47:00
3,108,0004247182025030620250612,2025-05-17,2025-05-17 17:15:00
4,112,0004247012025030620250612,2025-05-17,2025-05-17 09:44:00


In [226]:
print(sensors.shape)

(3376, 4)


In [227]:
len(sensors['trip_id'].unique())

549

In [228]:
len(tickets['trip_id'].unique())

549

In [229]:
conn.rollback()

query = """
SELECT *
FROM trips
"""
trips = pd.read_sql(query, conn)
trips.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id,wheelchair_accessible
0,607,0000001432024090920250612,0004187192024090920250612,Sorni,False,None,NaN
1,607,0000001442024090920250612,0004187202024090920250612,Pressano Piazza,False,None,NaN
2,607,0000001442024090920250612,0004187212024090920250612,Pressano Piazza,False,None,NaN
3,613,0000001482024090920250612,0004187222024090920250612,Pressano Piazza,False,None,NaN
4,610,0000001462024090920250612,0004187232024090920250612,Lavis Stazione Fs,False,None,NaN


In [230]:
trips.shape

(3613, 7)

In [231]:
merged_tickets = tickets.merge(sensors, on=['trip_id','timestamp']).merge(trips, on='trip_id')#.sort_values('trip_id')
merged_tickets.head()

,passengers_in,trip_id,school,hospital,peak_hour,day_x,timestamp,passengers_out,day_y,route_id,service_id,trip_headsign,direction_id,shape_id,wheelchair_accessible
0,514,0004188642024090920250612,0,0,1,2025-05-17,2025-05-17 17:33:00,80,2025-05-17,400,0000000102024090920250612,Oltrecastello,False,D593_T0501b_Andata_PRINCIPALE_sub2,1.0
1,460,0004188672024090920250612,0,0,1,2025-05-17,2025-05-17 20:33:00,108,2025-05-17,400,0000000102024090920250612,Oltrecastello,False,D593_T0501b_Andata_PRINCIPALE_sub2,1.0
2,208,0004188712024090920250612,0,0,1,2025-05-17,2025-05-17 09:00:00,34,2025-05-17,400,0000000102024090920250612,Piazza Dante Dogana,True,D170_F0512_Ritorno_sub1,1.0
3,472,0004188752024090920250612,0,0,1,2025-05-17,2025-05-17 11:15:00,108,2025-05-17,400,0000000022024090920250612,Piazza Dante Dogana,True,D170_F0512_Ritorno_sub1,1.0
4,528,0004188762024090920250612,0,0,1,2025-05-17,2025-05-17 12:15:00,108,2025-05-17,400,0000000022024090920250612,Piazza Dante Dogana,True,D170_F0512_Ritorno_sub1,1.0


In [232]:
merged_tickets.shape

(2559, 15)

In [233]:
conn.rollback()

query = """
select trip_id, timestamp, traffic_level, normal, traffic
from traffic
"""
traffic = pd.read_sql(query, conn)
traffic.head()

,trip_id,timestamp,traffic_level,normal,traffic
0,0004188492024090920250612,2025-05-17 13:49:00,no traffic/low,909,833
1,0004188682024090920250612,2025-05-17 12:26:00,no traffic/low,777,738
2,0004188692024090920250612,2025-05-17 06:52:00,no traffic/low,778,722
3,0004190132024090920250612,2025-05-17 07:04:00,no traffic/low,905,848
4,0004190342024090920250612,2025-05-17 07:42:00,no traffic/low,745,728


merge traffic with trips to get shape id for later merge

In [234]:
traffic_merged = traffic.merge(trips, on='trip_id')
traffic_merged.head()

,trip_id,timestamp,traffic_level,normal,traffic,route_id,service_id,trip_headsign,direction_id,shape_id,wheelchair_accessible
0,0004188492024090920250612,2025-05-17 13:49:00,no traffic/low,909,833,400,0000000102024090920250612,Oltrecastello,False,D593_T0501b_Andata_PRINCIPALE_sub2,1.0
1,0004188682024090920250612,2025-05-17 12:26:00,no traffic/low,777,738,400,0000000102024090920250612,Povo Polo Sociale Cap.,False,D599_T0521h_Andata_sub1,1.0
2,0004188692024090920250612,2025-05-17 06:52:00,no traffic/low,778,722,400,0000000022024090920250612,Piazza Dante Dogana,True,D170_F0512_Ritorno_sub1,1.0
3,0004190132024090920250612,2025-05-17 07:04:00,no traffic/low,905,848,404,0000000032024090920250612,Verona Palazzo Stella,False,D660_T0865_Andata_sub1,1.0
4,0004190342024090920250612,2025-05-17 07:42:00,no traffic/low,745,728,404,0000000032024090920250612,Piazza Dante Dogana,True,D179_F0842b_Ritorno_ALTA_sub1,1.0


In [235]:
len(traffic_merged['shape_id'].unique())

34

In [236]:
traffic_merged.shape

(34, 11)

Before aggregating with traffic we need to make sure that the shape_id is included inside the table so to do we need to join the table with the trips table on the trip_id column to get the shape_id column.

In [237]:
traffic_tickets = merged_tickets.merge(traffic_merged, on=['shape_id'], how='right')[['traffic_level', 'normal', 'traffic', 'passengers_in', 'trip_id_x', 'school', 'hospital', 'peak_hour', 'timestamp_x', 'passengers_out']]
traffic_tickets.head()

,traffic_level,normal,traffic,passengers_in,trip_id_x,school,hospital,peak_hour,timestamp_x,passengers_out
0,no traffic/low,909,833,514,0004188642024090920250612,0,0,1,2025-05-17 17:33:00,80
1,no traffic/low,909,833,460,0004188672024090920250612,0,0,1,2025-05-17 20:33:00,108
2,no traffic/low,909,833,106,0004193602024090920250612,0,0,0,2025-05-17 07:10:00,14
3,no traffic/low,909,833,428,0004206382024090920250612,0,0,1,2025-05-17 09:04:00,84
4,no traffic/low,909,833,486,0004206412024090920250612,0,0,1,2025-05-17 12:04:00,122


In [238]:

print(traffic_tickets.shape)

(2559, 10)


In [239]:
conn.rollback()

query = """
select *
from weather
"""
weather = pd.read_sql(query, conn)
weather = weather.rename(columns={'timestamp':'time'})
weather.head()

,measurement_id,time,latitude,longitude,weather_code,precipitation_probability,temperature,hour
0,3361,2025-05-23 18:31:00,46.04,11.119999,1,0.0,13.2,2025-05-16 00:00:00
1,3362,2025-05-23 18:31:00,46.04,11.119999,2,0.0,12.6,2025-05-16 01:00:00
2,3363,2025-05-23 18:31:00,46.04,11.119999,1,0.0,11.7,2025-05-16 02:00:00
3,3364,2025-05-23 18:31:00,46.04,11.119999,1,0.0,10.8,2025-05-16 03:00:00
4,3365,2025-05-23 18:31:00,46.04,11.119999,1,0.0,10.0,2025-05-16 04:00:00


In [240]:
traffic_tickets['timestamp_x'] = pd.to_datetime(traffic_tickets['timestamp_x'])
weather['hour'] = pd.to_datetime(weather['hour'])

traffic_tickets = traffic_tickets.sort_values('timestamp_x')
weather = weather.sort_values('hour')

In [241]:
merged = pd.merge_asof(
    traffic_tickets,
    weather[['hour', 'temperature', 'precipitation_probability', 'weather_code', 'latitude', 'longitude']],
    left_on='timestamp_x',
    right_on='hour',          # match on timestamp column
    direction='backward',     # can also use 'backward' or 'forward'
    tolerance=pd.Timedelta('1000h')  # optional: max time difference allowed
)

In [242]:
merged.shape

(2559, 16)

In [243]:
merged

,traffic_level,normal,traffic,passengers_in,trip_id_x,school,hospital,peak_hour,timestamp_x,passengers_out,hour,temperature,precipitation_probability,weather_code,latitude,longitude
0,no traffic/low,460,424,134,0004229912024090920250612,0,0,0,2025-05-17 06:23:00,34,2025-05-17 06:00:00,10.8,0.0,0,46.04,11.119999
1,no traffic/low,809,766,288,0004197522024090920250612,0,0,0,2025-05-17 06:25:00,38,2025-05-17 06:00:00,10.8,0.0,0,46.04,11.119999
2,no traffic/low,713,677,252,0004220122024090920250612,0,0,0,2025-05-17 06:42:00,64,2025-05-17 06:00:00,10.8,0.0,0,46.04,11.119999
3,no traffic/low,758,768,268,0004247872025030620250612,0,0,0,2025-05-17 07:07:00,36,2025-05-17 07:00:00,13.5,0.0,0,46.04,11.119999
4,no traffic/low,909,833,106,0004193602024090920250612,0,0,0,2025-05-17 07:10:00,14,2025-05-17 07:00:00,13.5,0.0,0,46.04,11.119999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2554,no traffic/low,778,722,412,0004189052024090920250612,0,0,1,2025-05-23 19:55:00,90,2025-05-22 23:00:00,15.1,55.0,61,46.04,11.119999
2555,no traffic/low,909,833,274,0004188552024090920250612,0,0,1,2025-05-23 20:13:00,34,2025-05-22 23:00:00,15.1,55.0,61,46.04,11.119999
2556,no traffic/low,778,722,242,0004188832024090920250612,0,0,1,2025-05-23 20:14:00,54,2025-05-22 23:00:00,15.1,55.0,61,46.04,11.119999
2557,no traffic/low,909,833,262,0004188672024090920250612,0,0,1,2025-05-23 20:31:00,48,2025-05-22 23:00:00,15.1,55.0,61,46.04,11.119999


In [244]:
merged.columns

Index(['traffic_level', 'normal', 'traffic', 'passengers_in', 'trip_id_x',
       'school', 'hospital', 'peak_hour', 'timestamp_x', 'passengers_out',
       'hour', 'temperature', 'precipitation_probability', 'weather_code',
       'latitude', 'longitude'],
      dtype='object')

In [245]:
conn.rollback()

query = """
select *
from events
where latitude is not null
"""

events = pd.read_sql(query, conn)
events = events.drop_duplicates(subset=['day_event'])

In [246]:
events['day_event'] = pd.to_datetime(events['day_event'])

In [247]:
# Ensure both columns are in datetime format (date only, no time)
merged['timestamp'] = merged['timestamp_x']
events['day_event'] = pd.to_datetime(events['day_event']).dt.date
merged['timestamp_x'] = pd.to_datetime(merged['timestamp_x']).dt.date  # or 'date' depending on column name


# Merge on the date
final_with_events = pd.merge(events, merged, left_on='day_event', right_on='timestamp_x', how='right')  # use 'left', 'right', or 'outer' if needed

In [248]:
final_with_events

,event_id,event_name,day_event,start_time,end_time,location_event,latitude_x,longitude_x,traffic_level,normal,...,peak_hour,timestamp_x,passengers_out,hour,temperature,precipitation_probability,weather_code,latitude_y,longitude_y,timestamp
0,233.0,PRIMAVERA IN MUSICA A POVO,2025-05-17,20:30:00,None,"Casa Sociale - Sala Agorà, via Don Tommaso Dal...",46.065633,11.153894,no traffic/low,460,...,0,2025-05-17,34,2025-05-17 06:00:00,10.8,0.0,0,46.04,11.119999,2025-05-17 06:23:00
1,233.0,PRIMAVERA IN MUSICA A POVO,2025-05-17,20:30:00,None,"Casa Sociale - Sala Agorà, via Don Tommaso Dal...",46.065633,11.153894,no traffic/low,809,...,0,2025-05-17,38,2025-05-17 06:00:00,10.8,0.0,0,46.04,11.119999,2025-05-17 06:25:00
2,233.0,PRIMAVERA IN MUSICA A POVO,2025-05-17,20:30:00,None,"Casa Sociale - Sala Agorà, via Don Tommaso Dal...",46.065633,11.153894,no traffic/low,713,...,0,2025-05-17,64,2025-05-17 06:00:00,10.8,0.0,0,46.04,11.119999,2025-05-17 06:42:00
3,233.0,PRIMAVERA IN MUSICA A POVO,2025-05-17,20:30:00,None,"Casa Sociale - Sala Agorà, via Don Tommaso Dal...",46.065633,11.153894,no traffic/low,758,...,0,2025-05-17,36,2025-05-17 07:00:00,13.5,0.0,0,46.04,11.119999,2025-05-17 07:07:00
4,233.0,PRIMAVERA IN MUSICA A POVO,2025-05-17,20:30:00,None,"Casa Sociale - Sala Agorà, via Don Tommaso Dal...",46.065633,11.153894,no traffic/low,909,...,0,2025-05-17,14,2025-05-17 07:00:00,13.5,0.0,0,46.04,11.119999,2025-05-17 07:10:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no traffic/low,778,...,1,2025-05-23,90,2025-05-22 23:00:00,15.1,55.0,61,46.04,11.119999,2025-05-23 19:55:00
2555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no traffic/low,909,...,1,2025-05-23,34,2025-05-22 23:00:00,15.1,55.0,61,46.04,11.119999,2025-05-23 20:13:00
2556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no traffic/low,778,...,1,2025-05-23,54,2025-05-22 23:00:00,15.1,55.0,61,46.04,11.119999,2025-05-23 20:14:00
2557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no traffic/low,909,...,1,2025-05-23,48,2025-05-22 23:00:00,15.1,55.0,61,46.04,11.119999,2025-05-23 20:31:00


In [249]:
final_with_events['event_dummy'] = final_with_events['day_event'].notna().astype(int)

In [250]:
selected_features = ['trip_id_x', 'timestamp', 'temperature', 'precipitation_probability', 'weather_code', 'normal', 'traffic', 'traffic_level', 'event_dummy', 'passengers_in', 'passengers_out']

In [ ]:
final_with_events[selected_features]

,trip_id_x,timestamp,temperature,precipitation_probability,weather_code,normal,traffic,traffic_level,event_dummy,passengers_in,passengers_out
0,0004229912024090920250612,2025-05-17 06:23:00,10.8,0.0,0,460,424,no traffic/low,1,134,34
1,0004197522024090920250612,2025-05-17 06:25:00,10.8,0.0,0,809,766,no traffic/low,1,288,38
2,0004220122024090920250612,2025-05-17 06:42:00,10.8,0.0,0,713,677,no traffic/low,1,252,64
3,0004247872025030620250612,2025-05-17 07:07:00,13.5,0.0,0,758,768,no traffic/low,1,268,36
4,0004193602024090920250612,2025-05-17 07:10:00,13.5,0.0,0,909,833,no traffic/low,1,106,14
...,...,...,...,...,...,...,...,...,...,...,...
2554,0004189052024090920250612,2025-05-23 19:55:00,15.1,55.0,61,778,722,no traffic/low,0,412,90
2555,0004188552024090920250612,2025-05-23 20:13:00,15.1,55.0,61,909,833,no traffic/low,0,274,34
2556,0004188832024090920250612,2025-05-23 20:14:00,15.1,55.0,61,778,722,no traffic/low,0,242,54
2557,0004188672024090920250612,2025-05-23 20:31:00,15.1,55.0,61,909,833,no traffic/low,0,262,48
